In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle 
import warnings 
warnings.filterwarnings("ignore")


In [5]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

In [6]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents/intents.json').read()
intents = json.loads(data_file)

In [7]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        
        documents.append((w, intent['tag']))
        
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [8]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [9]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /home/codespace/nltk_data...


[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [10]:
#lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

words = sorted(list(set(words)))

#sort classes
classes = sorted(list(set(classes)))

#documents = combination between patterns and intents
print (len(documents), "documents\n", documents, "\n")

#classes = intents[tags]
print (len(classes), "classes\n", classes, "\n")

#words = all words, vocabulary
print (len(words), "unique lemmatized words", words, "\n")

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

405 documents
 [(['Hi'], 'greeting'), (['How', 'are', 'you', '?'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['What', "'s", 'up'], 'greeting'), (['how', 'are', 'ya'], 'greeting'), (['heyy'], 'greeting'), (['whatsup'], 'greeting'), (['?', '?', '?', '?', '?', '?', '?', '?'], 'greeting'), (['cya'], 'goodbye'), (['see', 'you'], 'goodbye'), (['bye', 'bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['I', 'am', 'Leaving'], 'goodbye'), (['Bye'], 'goodbye'), (['Have', 'a', 'Good', 'day'], 'goodbye'), (['talk', 'to', 'you', 'later'], 'goodbye'), (['ttyl'], 'goodbye'), (['i', 'got', 'to', 'go'], 'goodbye'), (['gtg'], 'goodbye'), (['what', 'is', 'the', 'name', 'of', 'your', 'developers'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'your', 'creators'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'developers'], 'creator'), (['what', 'is', 'the', 'name', 'of', 'the', 'creato

In [11]:
#creating our training data 
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    
    # initialize our bag of words
    bag = []
    
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

# shuffle our features and turn into list
random.shuffle(training)

# create train and test lists. X - patterns, Y - intents
train_x = [pattern[0] for pattern in training]
train_y = [pattern[1] for pattern in training]
print("Training data created")

Training data created


In [12]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [13]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
print("First layer: ", model.layers[0].get_weights()[0])

First layer:  [[-0.0117376   0.00573441  0.12262025 ... -0.04524233 -0.02081519
   0.10360834]
 [-0.05672473  0.1072644  -0.0799308  ... -0.02966589  0.09440809
   0.11086798]
 [ 0.06335643 -0.06805027  0.0117311  ... -0.04010135  0.09748097
  -0.12284238]
 ...
 [-0.04328524  0.07522914 -0.05600902 ... -0.0734673   0.03637379
  -0.07587217]
 [ 0.07268213 -0.08610316 -0.07539445 ...  0.08154631 -0.01225079
  -0.10961052]
 [-0.02550994 -0.0126487   0.11784874 ...  0.01174475  0.01835427
   0.11397517]]


In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/100


81/81 [==============================] - 1s 1ms/step - loss: 3.6220 - accuracy: 0.0395
Epoch 2/100
81/81 [==============================] - 0s 1ms/step - loss: 3.4831 - accuracy: 0.1062
Epoch 3/100
81/81 [==============================] - 0s 1ms/step - loss: 3.2795 - accuracy: 0.1852
Epoch 4/100
81/81 [==============================] - 0s 1ms/step - loss: 3.0380 - accuracy: 0.2568
Epoch 5/100
81/81 [==============================] - 0s 2ms/step - loss: 2.7570 - accuracy: 0.2914
Epoch 6/100
81/81 [==============================] - 0s 1ms/step - loss: 2.5179 - accuracy: 0.3531
Epoch 7/100
81/81 [==============================] - 0s 1ms/step - loss: 2.2668 - accuracy: 0.4321
Epoch 8/100
81/81 [==============================] - 0s 1ms/step - loss: 2.0474 - accuracy: 0.4716
Epoch 9/100
81/81 [==============================] - 0s 2ms/step - loss: 1.8791 - accuracy: 0.5062
Epoch 10/100
81/81 [==============================] - 0s 1ms/step - loss: 1.7028 - accuracy: 0.5753
Epoch 11/100
81/81 [=

In [16]:
from keras.models import load_model
model = load_model('chatbot_model.h5')

In [25]:
intents = json.loads(open('/workspaces/Song_Rec/chatbot_test/intents/intents.json').read())

words = pickle.load(open('/workspaces/Song_Rec/chatbot_test/words.pkl','rb'))

classes = pickle.load(open('/workspaces/Song_Rec/chatbot_test/classes.pkl','rb'))

In [26]:
def clean_up_sentences(sentences):
    # tokenize the pattern - split words into array

    sentence_words = nltk.word_tokenize(sentence)
    #print(sentence_words)
    # stem each word - create short form for word

    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    #print(sentence_words)

    return sentence_words

In [27]:
def bow(sentence, words, show_details=True):

    # tokenize the pattern

    sentence_words = clean_up_sentence(sentence)
    #print(sentence_words)

    # bag of words - matrix of N words, vocabulary matrix

    bag = [0]*len(words) 
    #print(bag)

    for s in sentence_words:  
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
                #print ("found in bag: %s" % w)
    #print(bag)
    return(np.array(bag))

In [28]:
def predict_class(sentence, model):

    # filter out predictions below a threshold

    p = bow(sentence, words,show_details=False)
    #print(p)

    res = model.predict(np.array([p]))[0]
    #print(res)

    ERROR_THRESHOLD = 0.25

    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    #print(results)
    # sort by strength of probability

    results.sort(key=lambda x: x[1], reverse=True)
    #print(results)

    return_list = []

    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})

    return return_list
    #print(return_list)


In [29]:
def getResponse(ints, intents_json):

    tag = ints[0]['intent']
    #print(tag)

    list_of_intents = intents_json['intents']
    #print(list_of_intents)

    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result


In [31]:
def chatbot_response(text): 
    
    ints = predict_class(text, model)

    #print(ints)
    res = getResponse(ints, intents)
    #print(res)
    return res

In [32]:
start = True

while start:

    query = input('Enter Message:')
    if query in ['quit','exit','bye']:
        start = False
        continue
    try:
        res = chatbot_response(query)
        print(res)
    except:
        print('You may need to rephrase your question.')

You may need to rephrase your question.
You may need to rephrase your question.
You may need to rephrase your question.
You may need to rephrase your question.
